In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
import tqdm

# df = pd.read_feather("traning_imgs.df")
# old = pd.read_feather("../data_analysis/prelim.feather").set_index("key")
# df["seq_key"] = old.loc[df["key"].values,"sequence_key"].values
# df["view_direction"] = old.loc[df["key"].values,"view_direction"].values
# df["city"] = df.path.apply(lambda x: x[22: x.find("/",22)])
# func = lambda s: s[s.find("images/")+7:]
# df["local_path"] =  "../seq_data/" + df.ds_type + "/" + df.path.apply(func).values
# df = df.sort_values(["city","seq_key","og_cluster"])
# df.index = range(len(df))
# df.to_feather("traning_imgs.df")

c:\Users\ual\.conda\envs\felix\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# df = pd.read_feather("traning_imgs.df")

# model = tf.keras.models.load_model("../../view_directions_task_single_image/Training/training_results/last_model/ENB0_All/model")
# preds = model.predict(ds,verbose=1)
# def mapping(path,label):

#     image = tf.io.read_file(path)
#     image = tf.io.decode_jpeg(image, channels=3)
#     label = tf.one_hot(label,2)
#     return image, label
# ds = tf.data.Dataset.from_tensor_slices((df.local_path.values,(df.view_direction=="Sideways").apply(int).values)).map(mapping).batch(64)

# df["lon"] = preds[:,0]
# df["lat"] = preds[:,1]
# df.to_feather('imgs_preds.df')

In [3]:
y_true = [[0, 1], [1,0], [1,0]]
y_pred = [[-18.6, 0.51], [1,0], [1,0]]
bce = tf.keras.losses.BinaryCrossentropy(from_logits=False)
bce(y_true, y_pred).numpy().mean()


0.11222405

In [4]:
df = pd.read_feather('imgs_preds.df').sort_values(["city",'seq_key','og_cluster'])
labels = tf.one_hot((df.view_direction == "Sideways").apply(int).values,2).numpy()
preds = np.array([df.lon,df.lat]).T

df['label0'] = labels[:,0]
df['label1'] = labels[:,1]


df.head()

,key,cluster_id,ds_type,og_cluster,path,seq_key,city,local_path,view_direction,lon,lat,label0,label1
0,wdIMhVzUKwF-wgJxrhL0lQ,26,train,26,Places_Data/train_val/amman/query/images/wdIMh...,004Lc-M_TjGRo7Ve-0Fhqw,amman,../seq_data/train/wdIMhVzUKwF-wgJxrhL0lQ.jpg,Forward,0.999992,8.027482e-06,1.0,0.0
1,QdtKhFvv3cDcoHBArvcHbw,26,train,26,Places_Data/train_val/amman/query/images/QdtKh...,004Lc-M_TjGRo7Ve-0Fhqw,amman,../seq_data/train/QdtKhFvv3cDcoHBArvcHbw.jpg,Forward,1.000000,1.604972e-07,1.0,0.0
2,7RjQtvVfAdRd3qnm80w4Yg,26,train,26,Places_Data/train_val/amman/query/images/7RjQt...,004Lc-M_TjGRo7Ve-0Fhqw,amman,../seq_data/train/7RjQtvVfAdRd3qnm80w4Yg.jpg,Forward,0.999999,1.003697e-06,1.0,0.0
3,pW-u_bvOqxgZU6oGeyQjEA,26,train,26,Places_Data/train_val/amman/query/images/pW-u_...,004Lc-M_TjGRo7Ve-0Fhqw,amman,../seq_data/train/pW-u_bvOqxgZU6oGeyQjEA.jpg,Forward,0.999940,6.010981e-05,1.0,0.0
4,ritUetJDRquvOYH8Z5qbNQ,26,train,26,Places_Data/train_val/amman/query/images/ritUe...,004Lc-M_TjGRo7Ve-0Fhqw,amman,../seq_data/train/ritUetJDRquvOYH8Z5qbNQ.jpg,Forward,0.999498,5.021754e-04,1.0,0.0


In [5]:
index = []
vals = []
bce = tf.keras.losses.BinaryCrossentropy(from_logits=False)
for i in tqdm.tqdm(df.groupby(["city",'seq_key','og_cluster'])):

    preds = np.array([i[1].lon.values, i[1].lat.values]).T
    labels = np.array([i[1].label0.values, i[1].label1.values]).T
    
    loss = bce(labels,preds).numpy()
    vals.append(loss)
    index.append(i[0])

100%|██████████| 15325/15325 [00:29<00:00, 511.44it/s]


In [10]:
losses = pd.DataFrame(vals,index=pd.MultiIndex.from_tuples(index),columns=['loss'])
losses.sort_values("loss",ascending=False,inplace=True)
losses.reset_index().to_feather('losses.df')